In [3]:
import chromadb
from chromadb.config import Settings

# Configuración del cliente
client = chromadb.Client(Settings(persist_directory="./chromadb"))

# Crear una colección para los partidos de fútbol
collection = client.create_collection(name="partidos_futbol")


In [4]:
from sentence_transformers import SentenceTransformer

# Modelo de embeddings
modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

# Datos de partidos de fútbol
partidos = [
    {"id": "1", "texto": "Partido entre Barcelona y Real Madrid en el Camp Nou el 25 de diciembre de 2025.", "metadata": {"equipo_local": "Barcelona", "equipo_visitante": "Real Madrid", "fecha": "2025-12-25", "estadio": "Camp Nou"}},
    {"id": "2", "texto": "Partido entre Manchester United y Liverpool en Old Trafford el 10 de enero de 2025.", "metadata": {"equipo_local": "Manchester United", "equipo_visitante": "Liverpool", "fecha": "2025-01-10", "estadio": "Old Trafford"}},
    {"id": "3", "texto": "Partido entre Juventus y Inter de Milan en el Allianz Stadium el 5 de marzo de 2025.", "metadata": {"equipo_local": "Juventus", "equipo_visitante": "Inter de Milan", "fecha": "2025-03-05", "estadio": "Allianz Stadium"}},
]

# Generar embeddings para los textos
textos = [partido["texto"] for partido in partidos]
ids = [partido["id"] for partido in partidos]
metadatas = [partido["metadata"] for partido in partidos]
embeddings = modelo_embeddings.encode(textos).tolist()

# Insertar partidos con embeddings en la colección
collection.add(ids=ids, documents=textos, metadatas=metadatas, embeddings=embeddings)
print("Partidos con embeddings insertados.")


c:\Users\Usuario\vectorialDatabaseCrud\entorno\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Partidos con embeddings insertados.


In [7]:
# Generar embedding para la consulta
consulta = "Partido entre Barcelona y Real Madrid en el Camp Nou"
embedding_consulta = modelo_embeddings.encode([consulta]).tolist()

# Buscar los documentos más similares
resultados_similares = collection.query(query_embeddings=embedding_consulta, n_results=2)
print("Partidos similares:", resultados_similares)


Partidos similares: {'ids': [['1', '2']], 'embeddings': None, 'documents': [['Partido entre Barcelona y Real Madrid en el Camp Nou el 25 de diciembre de 2025, con 70,000 espectadores.', 'Partido entre Manchester United y Liverpool en Old Trafford el 10 de enero de 2025.']], 'uris': None, 'data': None, 'metadatas': [[{'equipo_local': 'Barcelona', 'equipo_visitante': 'Real Madrid', 'estadio': 'Camp Nou', 'fecha': '2025-12-25'}, {'equipo_local': 'Manchester United', 'equipo_visitante': 'Liverpool', 'estadio': 'Old Trafford', 'fecha': '2025-01-10'}]], 'distances': [[0.5082851648330688, 0.8440568447113037]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [6]:
# Actualizar un partido y su embedding
nuevo_texto = "Partido entre Barcelona y Real Madrid en el Camp Nou el 25 de diciembre de 2025, con 70,000 espectadores."
nuevo_embedding = modelo_embeddings.encode([nuevo_texto]).tolist()
collection.delete(ids=["1"])  # Eliminar partido antiguo
collection.add(ids=["1"], documents=[nuevo_texto], metadatas=[{"equipo_local": "Barcelona", "equipo_visitante": "Real Madrid", "fecha": "2025-12-25", "estadio": "Camp Nou"}], embeddings=nuevo_embedding)
print("Partido actualizado con nuevo embedding.")


Partido actualizado con nuevo embedding.


In [8]:
# Eliminar un partido
collection.delete(ids=["2"])
print("Partido eliminado.")


Partido eliminado.


In [ ]:
client.persist()

In [10]:
from IPython.display import clear_output
clear_output(wait=True)